In [1]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

class Cats_Vs_Dogs_Dataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = img_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.img_dir))

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(idx) + ".jpg")
        resize = transforms.Resize(size=(128, 128))
        image = resize(read_image(img_path))
        label = image
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [19]:
learning_rate = 0.01
batch_size = 16
num_epochs = 25

In [20]:
train_set = Cats_Vs_Dogs_Dataset(img_dir=r"E:\\College\\FCAI-4th Year\\First Term\\Generative Adversarial Networks\\Assginments\\Assginment 3\\train_modified\\train_small")
test_set = Cats_Vs_Dogs_Dataset(img_dir=r"E:\\College\\FCAI-4th Year\\First Term\\Generative Adversarial Networks\\Assginments\\Assginment 3\\test_modified\\test1")
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [21]:
def conv(ni, nf): return nn.Conv2d(ni, nf, kernel_size=3, stride=1, padding='same')
def pool(): return nn.MaxPool2d(kernel_size=2, stride=2)
def conv1x1(ni, nf): return nn.Conv2d(ni, nf, kernel_size=1, stride=1, padding='same')
def bn(ni): return nn.BatchNorm2d(ni)
def up(): return nn.Upsample(scale_factor=2)

In [22]:
def down_stage(ni, nf):
    return nn.Sequential(
        pool(),
        conv(ni, nf),
        nn.ReLU(inplace=True),
        bn(nf),
        conv(nf, nf),
        nn.ReLU(inplace=True)
    )

def up_stage(ni, nf):
    return nn.Sequential(
        up(),
        conv(ni, nf),
        nn.ReLU(inplace=True),
        bn(nf),
        conv(nf, nf),
        nn.ReLU(inplace=True)
    )

In [23]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
        conv(3, 32), # 128
        down_stage(32, 64), # 64
        down_stage(64, 128),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
        up_stage(128, 64),
        up_stage(64, 32), # 128
        conv(32, 16), # 128
        conv1x1(16, 3) # 128
        )
        
        self.test = False
    def forward(self, x):
        x = self.encoder(x)
        if self.test: x += torch.normal(0, 35, size=x.size)
        x = self.decoder(x)
        return x

In [24]:
model = Autoencoder()

In [25]:
from torchinfo import summary

model = model
batch_size = 16
summary(model, input_size=(batch_size, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
Autoencoder                              [16, 3, 128, 128]         --
├─Sequential: 1-1                        [16, 128, 32, 32]         --
│    └─Conv2d: 2-1                       [16, 32, 128, 128]        896
│    └─Sequential: 2-2                   [16, 64, 64, 64]          --
│    │    └─MaxPool2d: 3-1               [16, 32, 64, 64]          --
│    │    └─Conv2d: 3-2                  [16, 64, 64, 64]          18,496
│    │    └─ReLU: 3-3                    [16, 64, 64, 64]          --
│    │    └─BatchNorm2d: 3-4             [16, 64, 64, 64]          128
│    │    └─Conv2d: 3-5                  [16, 64, 64, 64]          36,928
│    │    └─ReLU: 3-6                    [16, 64, 64, 64]          --
│    └─Sequential: 2-3                   [16, 128, 32, 32]         --
│    │    └─MaxPool2d: 3-7               [16, 64, 32, 32]          --
│    │    └─Conv2d: 3-8                  [16, 128, 32, 32]         73,856
│

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
model = model.to(device)

In [28]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_loader), epochs=num_epochs)

In [29]:
torch.cuda.empty_cache()

In [57]:
outputs = []
for epoch in range(num_epochs):
    os.chdir(r'E:\\College\\FCAI-4th Year\\First Term\\Generative Adversarial Networks\\Assginments\\Assginment 3\\train_modified\\train')
    for batch_idx, (image, targets) in enumerate(tqdm(train_loader)):
        image = (image.float()).to(device=device)
        targets = (targets.float()).to(device=device)
        image += (torch.normal(0, 35, size=(3, 128, 128))).to(device=device)
        recon = model(image)
        loss = criterion(recon, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:32<00:00,  1.46it/s]


Epoch:1, Loss:389.6310


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:48<00:00,  1.02s/it]


Epoch:2, Loss:673.6208


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:48<00:00,  1.02s/it]


...

Epoch:23, Loss:1251.2225


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:48<00:00,  1.02s/it]


Epoch:24, Loss:406.2216


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:48<00:00,  1.02s/it]


Epoch:25, Loss:389.8959
